In [2]:
import matplotlib.pyplot as plt
import scipy.stats as sts
import scipy.signal as sigs
import numpy as np
import cmdstanpy ## import stan interface for Python
import os
import seaborn as sns
import pandas as pd
from importlib import reload 
import sys
sys.path.append("..")
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import arviz

import matplotlib as mpl
import warnings
warnings.filterwarnings("ignore")
from statannotations.Annotator import Annotator
from itertools import product

tex_fonts = {
    # Use LaTeX to write all text
#     "text.usetex": True,
    "font.family": "Helvetica",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 7,
    "font.size": 7,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 6,
    "xtick.labelsize": 6,
    "ytick.labelsize": 6,
    "axes.grid": True,
    'grid.color': '#DDDDDD',
    'grid.linestyle': '-',
    'grid.linewidth': 0.3,
    "lines.markersize":5,
    "lines.markeredgewidth":1,
    'axes.axisbelow':True,
    'pdf.fonttype':42,
    'axes.linewidth':0.5,
        'xtick.major.width':0.5,
    'ytick.major.width':0.5,
    'ytick.minor.width':0.3,

    'ytick.major.pad':0.2,
        "xtick.major.size":3,
    "ytick.major.size":3,
}

plt.rcParams.update(tex_fonts)

if os.name == "nt": ## adds compiler to path in Windows
    cmdstanpy.utils.cxx_toolchain_path() 
    
%config InlineBackend.close_figures=False # keep figures open in pyplot
#%config InlineBackend.print_figure_kwargs = {'bbox_inches':"tight", 'pad_inches':2}

np.random.seed(2101)


ratio= 2/(1+np.sqrt(5))

width = 520.344

kwags = {"wspace":0.2}

my_pal = {"4cm": "g", 
          "4em": "m", 
          "donor":"#a4e0ef", 
          "host":"#ff7f0e",
         "WT":"#02feff",
         "Young":"#7d81fc",
         "Old":"#f50aff"}

my_marks = {"donor": "o", 
           "host": "X",
         "WT":"o",
         "Young CHIM":"P",
         "Old CHIM":"^"}

scatterkwags = {"palette": my_pal,
                  "markers":my_marks,
                  'edgecolor':"k",
}


def set_size(width, fraction=1, subplots=(1, 1)):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    #golden_ratio = (5**.5 - 1) / 2
    golden_ratio = 1

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

#loading dataframes for parameters and data
df_para = pd.read_csv('/home/elise/Code/BRDU/parameters_est.csv')
df = pd.read_csv('/home/elise/Code/BRDU/Elisehasbeenusing_dataUCL.csv',index_col='index')
df['age_cat']=np.where(df.age < 114, 'wt', pd.np.where((df.age >= 114) & (df.age < 180), 'Young', 'Old'))

def enumerated_product(*args):
    yield from zip(product(*(range(len(x)) for x in args)), product(*args))

def myMAP(data):
    
#     counts, x = np.histogram(data,bins=(np.int(np.rint(np.sqrt(len(data))))))
#     x_ind = np.unravel_index(np.argmax(counts), counts.shape)
# #     arrayed = data.to_numpy()

# #     i_peaks, _ = sigs.find_peaks(arrayed)

# #     i_peaks
# #     arrayed[i_peaks]

# #     # Find the index from the maximum peak
# #     i_max_peak = i_peaks[np.argmax(_['widths'])]

# #     # Find the x value from that index
# #     x_max = arrayed[i_max_peak]


# #     # Find the x value from that index
# #     x_max = arrayed[i_max_peak]
#     return x[x_ind]

    arrayed = data.to_numpy()
    nparam_density = sts.kde.gaussian_kde(arrayed)
    x = np.linspace(np.min(arrayed), np.max(arrayed), 2000)
    nparam_density = nparam_density(x)

    return x[np.argsort(nparam_density)[-1]]

def myMAP_CI(datad, datah):
    
#     counts, x = np.histogram(data,bins=(np.int(np.rint(np.sqrt(len(data))))))
#     x_ind = np.unravel_index(np.argmax(counts), counts.shape)
# #     arrayed = data.to_numpy()

# #     i_peaks, _ = sigs.find_peaks(arrayed)

# #     i_peaks
# #     arrayed[i_peaks]

# #     # Find the index from the maximum peak
# #     i_max_peak = i_peaks[np.argmax(_['widths'])]

# #     # Find the x value from that index
# #     x_max = arrayed[i_max_peak]


# #     # Find the x value from that index
# #     x_max = arrayed[i_max_peak]
#     return x[x_ind]

    arrayedd = datad.to_numpy()
    nparam_densityd = sts.kde.gaussian_kde(arrayedd)
    xd = np.linspace(np.min(arrayedd), np.max(arrayedd), 2000)
    nparam_densityd = nparam_densityd(xd)

    lA_calcd, uA_calcd = np.percentile(datad, q=[5, 95])
    
    arrayedh = datah.to_numpy()
    nparam_densityh = sts.kde.gaussian_kde(arrayedh)
    xh = np.linspace(np.min(arrayedh), np.max(arrayedh), 2000)
    nparam_densityh = nparam_densityh(xh)

    lA_calch, uA_calch = np.percentile(datah, q=[5, 95])
    
    diff = datad-datah
    arrayedf = diff.to_numpy()
    nparam_densityf = sts.kde.gaussian_kde(arrayedf)
    xf = np.linspace(np.min(arrayedf), np.max(arrayedf), 2000)
    nparam_densityf = nparam_densityf(xf)

    lA_calcf, uA_calcf = np.percentile(diff, q=[5, 95])
    
    return [xd[np.argsort(nparam_densityd)[-1]],  lA_calcd, uA_calcd, xh[np.argsort(nparam_densityh)[-1]],  lA_calch, uA_calch, xf[np.argsort(nparam_densityf)[-1]],  lA_calcf, uA_calcf] 




In [4]:

youngfile = '/opt/mesh/tiree/elise/samples_1/youngmice_r17/stan-cache-'
oldfile = '/opt/mesh/tiree/elise/samples_1/oldmice_r25/stan-cache-'

# parnames = ["alpha_A","alpha_B","delta_A","delta_B","beta","Source","mu","eff"] 
# pretty_parnames = ["$\\alpha$_A","$\\alpha$_B","$\\delta$_A", "$\\delta$_B","$\\beta$","Source","$\\mu$","$\\epsilon$"] #

parnames_l = ["alpha_A","alpha_B","delta_A","delta_B","beta","mu","eff","gamma"] 
pretty_parnames_l = ["$\\alpha$_A","$\\alpha$_B","$\\delta$_A", "$\\delta$_B","$\\beta$","$\\mu$","$\\epsilon$","$\\gamma$"] #

parnames_b = ["alpha_A","alpha_B","delta_A","delta_B","beta","mu","eff","fs"] 
pretty_parnames_b = ["$\\alpha$_A","$\\alpha$_B","$\\delta$_A", "$\\delta$_B","$\\beta$","$\\mu$","$\\epsilon$","fs"] #

pardf = pd.DataFrame()

stats = pd.DataFrame()


ADHc = ['host','donor']
populationc = ['4cm','4em']
agec = ['Young CHIM','Old CHIM']
typec = ['linear','branched']

for idx, adhpop in enumerated_product(ADHc,populationc, agec,typec):

    
    fl = []
    fb = []
    
    #data slicing and manulipulation
    if adhpop[2]=='Young CHIM':
        location = youngfile+adhpop[1]+adhpop[0].lower()+'_1/'
    else:
        location = oldfile+adhpop[1]+adhpop[0].lower()+'_1/'

    sys.path.insert(1, location)
    import paras
    reload(paras)
    
    for f_name in os.listdir(location):
        if f_name.endswith('.csv')&f_name.startswith('branched'):
            print(f_name)
            fb.append(location+f_name)
        elif f_name.endswith('.csv')&f_name.startswith('linear'):
            print(f_name)
            fl.append(location+f_name)

    if adhpop[3] == "branched":
        sam = cmdstanpy.from_csv(fb)
        
        k_hat_fl =  sam.stan_variable("k_hat")
        kihi_fl =  sam.stan_variable("f_kihi_calc")
        parests_flvl = [sam.stan_variable(pn) for pn in parnames_b]
        pardf_lvl=pd.DataFrame(np.transpose(parests_flvl), columns = parnames_b)
        pardf_lvl["Frac_Source_A"] = pardf_lvl["fs"]
    else:
        sam = cmdstanpy.from_csv(fl)
        
        k_hat_fl =  sam.stan_variable("k_hat")
        kihi_fl =  sam.stan_variable("f_kihi_calc")
        parests_flvl = [sam.stan_variable(pn) for pn in parnames_l]
        pardf_lvl=pd.DataFrame(np.transpose(parests_flvl), columns = parnames_l)
#         pardf_lvl["Source_B"] = pardf_lvl["gamma"]*np.mean(np.sum(k_hat_fl[:,:,0:paras.switch],axis=2), axis=1)
        
        pardf_lvl["fastcellsurvival"] = abs(pardf_lvl["gamma"].div(pardf_lvl["gamma"]+pardf_lvl["delta_A"]))
        pardf_lvl["averagetimeinfast"]=1/(pardf_lvl["gamma"]+pardf_lvl["delta_A"])
        pardf_lvl["clonallifetimeinfast"]=1/(pardf_lvl["gamma"]+pardf_lvl["delta_A"]-pardf_lvl["alpha_A"])
        pardf_lvl["Frac_Source_A"] = np.random.normal(1,0.001, len(k_hat_fl))
        

    pardf_lvl['type']=adhpop[3]
    pardf_lvl['population']=paras.populationc
    pardf_lvl['doh']=paras.ADHc
    pardf_lvl['popdoh']=paras.populationc+paras.ADHc
    pardf_lvl['ones']=1
    pardf_lvl['age']=adhpop[2]
    pardf_lvl['alpha_adata']=1/paras.alpha_A
    pardf_lvl['alpha_bdata']=1/paras.alpha_B
    pardf_lvl['delta_adata']=1/paras.delta_A
    pardf_lvl['delta_bdata']=1/paras.delta_B
    pardf_lvl['betadata']=1/paras.beta
    pardf_lvl['effdata']=paras.eff
    pardf_lvl['mudata']=paras.mu
    pardf_lvl['sourcedata']=paras.SourceL
    pardf_lvl['counts_t0']=paras.counts_t0
    pardf_lvl['FastSlowRatio']=np.mean(np.sum(k_hat_fl[:,:,0:paras.switch],axis=2), axis=1)/np.mean(np.sum(k_hat_fl[:,:,paras.switch:],axis=2), axis=1)
    pardf_lvl['FastFraction']=np.mean(np.sum(k_hat_fl[:,:,0:paras.switch],axis=2), axis=1)/np.mean(np.sum(k_hat_fl[:,:,:],axis=2), axis=1)

    pardf_lvl["inv_alpha_A"] = pardf_lvl["ones"].div(pardf_lvl["alpha_A"].values)
    pardf_lvl["inv_alpha_B"] = pardf_lvl["ones"].div(pardf_lvl["alpha_B"].values)
    pardf_lvl["inv_delta_A"] = pardf_lvl["ones"].div(pardf_lvl["delta_A"].values)
    pardf_lvl["inv_delta_B"] = pardf_lvl["ones"].div(pardf_lvl["delta_B"].values)
    pardf_lvl["inv_beta"] = pardf_lvl["ones"].div(pardf_lvl["beta"].values)
#     pardf_lvl["frac_source"] = pardf_lvl["Source"].div(pardf_lvl["counts_t0"].values)
    pardf_lvl["lambda_A"] = pardf_lvl["delta_A"] - pardf_lvl["alpha_A"]
    pardf_lvl["lambda_B"] = pardf_lvl["delta_B"] - pardf_lvl["alpha_B"]
    pardf_lvl["ratio_lambda"] = pardf_lvl["lambda_A"]/(pardf_lvl["lambda_B"])
    pardf_lvl['KihiFrac'] = np.mean(kihi_fl,axis=1)
    pardf_lvl['lifespanslow'] = np.log(2)/(pardf_lvl["lambda_B"])


#     pardf = pd.concat([pardf,pardf_lvl, pardf_bvl])
    pardf = pd.concat([pardf,pardf_lvl])

stats = pd.DataFrame()

for idx, adhpop in enumerated_product(populationc, agec):

    names_b = ["Frac_Source_A","inv_alpha_A","inv_delta_A","inv_alpha_B","inv_delta_B","eff","mu","inv_beta","FastFraction","lambda_A","lambda_B","ratio_lambda","KihiFrac","lifespanslow"]
    names_l = ["gamma","inv_alpha_A","inv_delta_A","inv_alpha_B","inv_delta_B","eff","mu","inv_beta","FastFraction","lambda_A","lambda_B","ratio_lambda","KihiFrac","lifespanslow"]

    type_i = 'branched'
    if type_i == "branched":
        stats_lvl=pd.DataFrame(data=[myMAP_CI(pardf[(pardf['type']==type_i)&(pardf['age']==adhpop[1])&(pardf['popdoh']==adhpop[0]+'donor')][i],pardf[(pardf['type']==type_i)&(pardf['age']==adhpop[1])&(pardf['popdoh']==adhpop[0]+'host')][i]) for i in names_b], 
                           columns=['D_MAP','D_l_CI','D_u_CI','H_MAP','H_l_CI','H_u_CI','dif_MAP','dif_l_CI','dif_u_CI'],index=names_b)
    else:
        stats_lvl=pd.DataFrame(data=[myMAP_CI(pardf[(pardf['type']==type_i)&(pardf['age']==adhpop[1])&(pardf['popdoh']==adhpop[0]+'donor')][i],pardf[(pardf['type']==type_i)&(pardf['age']==adhpop[1])&(pardf['popdoh']==adhpop[0]+'host')][i]) for i in names_l], 
                           columns=['D_MAP','D_l_CI','D_u_CI','H_MAP','H_l_CI','H_u_CI','dif_MAP','dif_l_CI','dif_u_CI'],index=names_b)
 
    stats_lvl['population']=adhpop[0]
    stats_lvl['age']=adhpop[1]
    cols = stats_lvl.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    cols = cols[-1:] + cols[:-1]
    stats_lvl = stats_lvl[cols]
    stats_lvl = stats_lvl.reset_index()
    
    stats = pd.concat([stats,stats_lvl])
    
    
stats = stats.sort_index()


stats.to_csv('/home/elise/Code/BRDU/parameters'+type_i+'.csv')


linearrealyoung-20230911134243_2.csv
branchedrealyoung-20230911134243_4.csv
branchedrealyoung-20230911134243_2.csv
linearrealyoung-20230911134243_4.csv
linearrealyoung-20230911134243_1.csv
branchedrealyoung-20230911134243_5.csv
branchedrealyoung-20230911134243_1.csv
linearrealyoung-20230911134243_3.csv
branchedrealyoung-20230911134243_3.csv
linearrealyoung-20230911134243_5.csv
linearrealyoung-20230911134243_2.csv
branchedrealyoung-20230911134243_4.csv
branchedrealyoung-20230911134243_2.csv
linearrealyoung-20230911134243_4.csv
linearrealyoung-20230911134243_1.csv
branchedrealyoung-20230911134243_5.csv
branchedrealyoung-20230911134243_1.csv
linearrealyoung-20230911134243_3.csv
branchedrealyoung-20230911134243_3.csv
linearrealyoung-20230911134243_5.csv
linearrealold-20230912111553_5.csv
branchedrealold-20230912111553_3.csv
branchedrealold-20230912111553_2.csv
linearrealold-20230912111553_4.csv
branchedrealold-20230912111553_4.csv
linearrealold-20230912111553_2.csv
linearrealold-2023091211

In [3]:

youngfile = '/opt/mesh/tiree/elise/samples_1/youngmice_r19/stan-cache-'
oldfile = '/opt/mesh/tiree/elise/samples_1/oldmice_r26/stan-cache-'

# parnames = ["alpha_A","alpha_B","delta_A","delta_B","beta","Source","mu","eff"] 
# pretty_parnames = ["$\\alpha$_A","$\\alpha$_B","$\\delta$_A", "$\\delta$_B","$\\beta$","Source","$\\mu$","$\\epsilon$"] #

parnames_l = ["alpha_A","alpha_B","delta_A","delta_B","beta","mu","eff","gamma"] 
pretty_parnames_l = ["$\\alpha$_A","$\\alpha$_B","$\\delta$_A", "$\\delta$_B","$\\beta$","$\\mu$","$\\epsilon$","$\\gamma$"] #

parnames_b = ["alpha_A","alpha_B","delta_A","delta_B","beta","mu","eff","fs"] 
pretty_parnames_b = ["$\\alpha$_A","$\\alpha$_B","$\\delta$_A", "$\\delta$_B","$\\beta$","$\\mu$","$\\epsilon$","fs"] #

pardf = pd.DataFrame()

stats = pd.DataFrame()


ADHc = ['host','donor']
populationc = ['4cm','4em']
agec = ['Young CHIM','Old CHIM']
typec = ['linear','branched',"burst"]

for idx, adhpop in enumerated_product(ADHc,populationc, agec,typec):

    
    fl = []
    fb = []
    fs = []
    
    #data slicing and manulipulation
    if adhpop[2]=='Young CHIM':
        location = youngfile+adhpop[1]+adhpop[0].lower()+'_1/'
    else:
        location = oldfile+adhpop[1]+adhpop[0].lower()+'_1/'

    sys.path.insert(1, location)
    import paras
    reload(paras)
    
    for f_name in os.listdir(location):
        if f_name.endswith('.csv')&f_name.startswith('branched'):
            print(f_name)
            fb.append(location+f_name)
        elif f_name.endswith('.csv')&f_name.startswith('linear'):
            print(f_name)
            fl.append(location+f_name)
        elif f_name.endswith('.csv')&f_name.startswith('burst'):
            print(f_name)
            fs.append(location+f_name)

    if adhpop[3] == "branched":
        sam = cmdstanpy.from_csv(fb)
        
        k_hat_fl =  sam.stan_variable("k_hat")
        kihi_fl =  sam.stan_variable("f_kihi_calc")
        parests_flvl = [sam.stan_variable(pn) for pn in parnames_b]
        pardf_lvl=pd.DataFrame(np.transpose(parests_flvl), columns = parnames_b)
        pardf_lvl["Frac_Source_A"] = pardf_lvl["fs"]
    elif adhpop[3] == "burst":
        sam = cmdstanpy.from_csv(fs)
        
        k_hat_fl =  sam.stan_variable("k_hat")
        kihi_fl =  sam.stan_variable("f_kihi_calc")
        parests_flvl = [sam.stan_variable(pn) for pn in parnames_l]
        pardf_lvl=pd.DataFrame(np.transpose(parests_flvl), columns = parnames_l)        
        pardf_lvl["fastcellsurvival"] = abs(pardf_lvl["gamma"].div(pardf_lvl["gamma"]+pardf_lvl["delta_A"]))
        pardf_lvl["averagetimeinfast"]=1/(pardf_lvl["gamma"]+pardf_lvl["delta_A"])
        pardf_lvl["clonallifetimeinfast"]=1/(pardf_lvl["gamma"]+pardf_lvl["delta_A"]-pardf_lvl["alpha_A"])
        pardf_lvl["Frac_Source_A"] = np.random.normal(1,0.001, len(k_hat_fl))
    else:
        sam = cmdstanpy.from_csv(fl)
        
        k_hat_fl =  sam.stan_variable("k_hat")
        kihi_fl =  sam.stan_variable("f_kihi_calc")
        parests_flvl = [sam.stan_variable(pn) for pn in parnames_l]
        pardf_lvl=pd.DataFrame(np.transpose(parests_flvl), columns = parnames_l)
#         pardf_lvl["Source_B"] = pardf_lvl["gamma"]*np.mean(np.sum(k_hat_fl[:,:,0:paras.switch],axis=2), axis=1)
        
        pardf_lvl["fastcellsurvival"] = abs(pardf_lvl["gamma"].div(pardf_lvl["gamma"]+pardf_lvl["delta_A"]))
        pardf_lvl["averagetimeinfast"]=1/(pardf_lvl["gamma"]+pardf_lvl["delta_A"])
        pardf_lvl["clonallifetimeinfast"]=1/(pardf_lvl["gamma"]+pardf_lvl["delta_A"]-pardf_lvl["alpha_A"])
        pardf_lvl["Frac_Source_A"] = np.random.normal(1,0.001, len(k_hat_fl))
        

    pardf_lvl['type']=adhpop[3]
    pardf_lvl['population']=paras.populationc
    pardf_lvl['doh']=paras.ADHc
    pardf_lvl['popdoh']=paras.populationc+paras.ADHc
    pardf_lvl['ones']=1
    pardf_lvl['age']=adhpop[2]
    pardf_lvl['alpha_adata']=1/paras.alpha_A
    pardf_lvl['alpha_bdata']=1/paras.alpha_B
    pardf_lvl['delta_adata']=1/paras.delta_A
    pardf_lvl['delta_bdata']=1/paras.delta_B
    pardf_lvl['betadata']=1/paras.beta
    pardf_lvl['effdata']=paras.eff
    pardf_lvl['mudata']=paras.mu
    pardf_lvl['sourcedata']=paras.SourceL
    pardf_lvl['counts_t0']=paras.counts_t0
    pardf_lvl['FastSlowRatio']=np.mean(np.sum(k_hat_fl[:,:,0:paras.switch],axis=2), axis=1)/np.mean(np.sum(k_hat_fl[:,:,paras.switch:],axis=2), axis=1)
    pardf_lvl['FastFraction']=np.mean(np.sum(k_hat_fl[:,:,0:paras.switch],axis=2), axis=1)/np.mean(np.sum(k_hat_fl[:,:,:],axis=2), axis=1)

    pardf_lvl["inv_alpha_A"] = pardf_lvl["ones"].div(pardf_lvl["alpha_A"].values)
    pardf_lvl["inv_alpha_B"] = pardf_lvl["ones"].div(pardf_lvl["alpha_B"].values)
    pardf_lvl["inv_delta_A"] = pardf_lvl["ones"].div(pardf_lvl["delta_A"].values)
    pardf_lvl["inv_delta_B"] = pardf_lvl["ones"].div(pardf_lvl["delta_B"].values)
    pardf_lvl["inv_beta"] = pardf_lvl["ones"].div(pardf_lvl["beta"].values)
#     pardf_lvl["frac_source"] = pardf_lvl["Source"].div(pardf_lvl["counts_t0"].values)
    pardf_lvl["lambda_A"] = pardf_lvl["delta_A"] - pardf_lvl["alpha_A"]
    pardf_lvl["lambda_B"] = pardf_lvl["delta_B"] - pardf_lvl["alpha_B"]
    pardf_lvl["ratio_lambda"] = pardf_lvl["lambda_A"]/(pardf_lvl["lambda_B"])
    pardf_lvl['KihiFrac'] = np.mean(kihi_fl,axis=1)
    pardf_lvl['lifespanslow'] = np.log(2)/(pardf_lvl["lambda_B"])



#     pardf = pd.concat([pardf,pardf_lvl, pardf_bvl])
    pardf = pd.concat([pardf,pardf_lvl])

stats = pd.DataFrame()

for idx, adhpop in enumerated_product(populationc, agec):

    names_b = ["Frac_Source_A","inv_alpha_A","inv_delta_A","inv_alpha_B","inv_delta_B","eff","mu","inv_beta","FastFraction","lambda_A","lambda_B","ratio_lambda","KihiFrac","lifespanslow"]
    names_l = ["gamma","inv_alpha_A","inv_delta_A","inv_alpha_B","inv_delta_B","eff","mu","inv_beta","FastFraction","lambda_A","lambda_B","ratio_lambda","KihiFrac","lifespanslow"]

    type_i = 'burst'
    if type_i == "branched":
        stats_lvl=pd.DataFrame(data=[myMAP_CI(pardf[(pardf['type']==type_i)&(pardf['age']==adhpop[1])&(pardf['popdoh']==adhpop[0]+'donor')][i],pardf[(pardf['type']==type_i)&(pardf['age']==adhpop[1])&(pardf['popdoh']==adhpop[0]+'host')][i]) for i in names_b], 
                           columns=['D_MAP','D_l_CI','D_u_CI','H_MAP','H_l_CI','H_u_CI','dif_MAP','dif_l_CI','dif_u_CI'],index=names_b)
    else:
        stats_lvl=pd.DataFrame(data=[myMAP_CI(pardf[(pardf['type']==type_i)&(pardf['age']==adhpop[1])&(pardf['popdoh']==adhpop[0]+'donor')][i],pardf[(pardf['type']==type_i)&(pardf['age']==adhpop[1])&(pardf['popdoh']==adhpop[0]+'host')][i]) for i in names_l], 
                           columns=['D_MAP','D_l_CI','D_u_CI','H_MAP','H_l_CI','H_u_CI','dif_MAP','dif_l_CI','dif_u_CI'],index=names_l)
    
    stats_lvl['population']=adhpop[0]
    stats_lvl['age']=adhpop[1]
    cols = stats_lvl.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    cols = cols[-1:] + cols[:-1]
    stats_lvl = stats_lvl[cols]
    stats_lvl = stats_lvl.reset_index()
    
    stats = pd.concat([stats,stats_lvl])
    
    
stats = stats.sort_index()


stats.to_csv('/home/elise/Code/BRDU/parameters'+type_i+'.csv')


linear-20240418181155_4.csv
branched-20240418181307_1.csv
branched-20240418181307_4.csv
burst-20240418181306_4.csv
branched-20240418181307_3.csv
burst-20240418181306_3.csv
linear-20240418181155_1.csv
linear-20240418181155_2.csv
burst-20240418181306_1.csv
burst-20240418181306_5.csv
branched-20240418181307_5.csv
burst-20240418181306_2.csv
linear-20240418181155_5.csv
linear-20240418181155_3.csv
branched-20240418181307_2.csv
linear-20240418181155_4.csv
branched-20240418181307_1.csv
branched-20240418181307_4.csv
burst-20240418181306_4.csv
branched-20240418181307_3.csv
burst-20240418181306_3.csv
linear-20240418181155_1.csv
linear-20240418181155_2.csv
burst-20240418181306_1.csv
burst-20240418181306_5.csv
branched-20240418181307_5.csv
burst-20240418181306_2.csv
linear-20240418181155_5.csv
linear-20240418181155_3.csv
branched-20240418181307_2.csv
linear-20240418181155_4.csv
branched-20240418181307_1.csv
branched-20240418181307_4.csv
burst-20240418181306_4.csv
branched-20240418181307_3.csv
burst

linear-20240418181325_5.csv
linear-20240418181325_1.csv
branched-20240418181355_4.csv
linear-20240418181325_4.csv
burst-20240418181355_2.csv
burst-20240418181355_5.csv
linear-20240418181325_2.csv
branched-20240418181355_1.csv
burst-20240418181355_1.csv
branched-20240418181355_2.csv
branched-20240418181355_3.csv
branched-20240418181355_5.csv
linear-20240418181325_3.csv
burst-20240418181355_4.csv
burst-20240418181355_3.csv
linear-20240418183027_2.csv
burst-20240418183056_4.csv
branched-20240418183056_5.csv
branched-20240418183056_1.csv
burst-20240418183056_3.csv
linear-20240418183027_3.csv
linear-20240418183027_4.csv
linear-20240418183027_1.csv
branched-20240418183056_2.csv
branched-20240418183056_3.csv
burst-20240418183056_5.csv
burst-20240418183056_1.csv
burst-20240418183056_2.csv
linear-20240418183027_5.csv
branched-20240418183056_4.csv
linear-20240418183027_2.csv
burst-20240418183056_4.csv
branched-20240418183056_5.csv
branched-20240418183056_1.csv
burst-20240418183056_3.csv
linear-2

In [17]:

youngfile = '/opt/mesh/tiree/elise/samples_1/youngmice_r17/stan-cache-'
oldfile = '/opt/mesh/tiree/elise/samples_1/oldmice_r25/stan-cache-'

youngfile_s = '/opt/mesh/tiree/elise/samples_1/youngmice_r19/stan-cache-'
oldfile_s = '/opt/mesh/tiree/elise/samples_1/oldmice_r26/stan-cache-'


ADHc = ['host','donor']
populationc = ['4cm','4em']
agec = ['Young CHIM','Old CHIM']
typec = ['linear','branched']
typec_b = ['linear','branched','burst']

# ADHc = ['donor']
# populationc = ['4cm']
agec = ['Old CHIM']

for idx, adhpop in enumerated_product(ADHc,populationc, agec):

        
    fl = []
    fb = []
    fs = []
    
    #data slicing and manulipulation
    if adhpop[2]=='Young CHIM':
        location = youngfile+adhpop[1]+adhpop[0].lower()+'_1/'
        location_s = youngfile_s+adhpop[1]+adhpop[0].lower()+'_1/'

    else:
        location = oldfile+adhpop[1]+adhpop[0].lower()+'_1/'
        location_s = oldfile_s+adhpop[1]+adhpop[0].lower()+'_1/'

    sys.path.insert(1, location)
    
    for f_name in os.listdir(location):
        if f_name.endswith('.csv')&f_name.startswith('branched'):
#             print(f_name)
            fb.append(location+f_name)
        elif f_name.endswith('.csv')&f_name.startswith('linear'):
#             print(f_name)
            fl.append(location+f_name)
            
    sys.path.insert(1, location_s)
    for f_name in os.listdir(location_s):
        if f_name.endswith('.csv')&f_name.startswith('burst'):
#             print(f_name)
            fs.append(location_s+f_name)
            
    sam_b = cmdstanpy.from_csv(fb)
    sam_l = cmdstanpy.from_csv(fl)
    sam_s = cmdstanpy.from_csv(fs)
    
    ardata_fb=arviz.from_cmdstanpy(#sam,
    posterior=sam_b,
    # prior=sam_pB,
    posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
        log_likelihood="log_lik",
    )

    ardata_fl=arviz.from_cmdstanpy(#sam,
        posterior=sam_l,
        # prior=sam_pL,
        posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
        log_likelihood="log_lik",
    )

    ardata_fs=arviz.from_cmdstanpy(#sam,
        posterior=sam_s,
        # prior=sam_pL,
        posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
        log_likelihood="log_lik",
    )

    compare_dict_linear = {"Linear": ardata_fl, "Branched": ardata_fb,"Burst":ardata_fs}

    print(adhpop)
    results = arviz.compare(compare_dict_linear)
    print(results.round(2))

#     fig1 = arviz.plot_compare(arviz.compare(compare_dict_linear)) #backend_kwargs = dict(figsize=(6,7)



('host', '4cm', 'Old CHIM')
          rank  elpd_loo  p_loo  elpd_diff  weight     se   dse  warning scale
Linear       0   -720.41   6.48       0.00    0.97  33.75  0.00    False   log
Branched     1   -720.49   6.53       0.08    0.03  33.79  0.41    False   log
Burst        2   -724.62   7.10       4.21    0.00  36.98  3.87    False   log
('host', '4em', 'Old CHIM')
          rank  elpd_loo  p_loo  elpd_diff  weight     se   dse  warning scale
Branched     0   -919.97   5.06       0.00    0.74  40.30  0.00    False   log
Linear       1   -920.13   4.97       0.16    0.00  40.19  0.59    False   log
Burst        2   -924.16   5.91       4.19    0.26  43.87  4.51    False   log
('donor', '4cm', 'Old CHIM')
          rank  elpd_loo  p_loo  elpd_diff  weight     se   dse  warning scale
Branched     0   -806.53   5.04       0.00    0.59  39.32  0.00    False   log
Linear       1   -806.54   5.34       0.02    0.37  39.30  0.37    False   log
Burst        2   -813.66   5.99       7.13    

In [6]:
print(fb)

['/opt/mesh/tiree/elise/samples_1/youngmice_r17/stan-cache-4emhost_1/branchedrealyoung-20230911134313_4.csv', '/opt/mesh/tiree/elise/samples_1/youngmice_r17/stan-cache-4emhost_1/branchedrealyoung-20230911134313_2.csv', '/opt/mesh/tiree/elise/samples_1/youngmice_r17/stan-cache-4emhost_1/branchedrealyoung-20230911134313_1.csv', '/opt/mesh/tiree/elise/samples_1/youngmice_r17/stan-cache-4emhost_1/branchedrealyoung-20230911134313_5.csv', '/opt/mesh/tiree/elise/samples_1/youngmice_r17/stan-cache-4emhost_1/branchedrealyoung-20230911134313_3.csv']


In [10]:

compare_dict_linear = {"Linear": ardata_fl, "Branched": ardata_fb,}


results = arviz.compare(compare_dict_linear)
print(results.round(2))


          rank  elpd_loo  p_loo  elpd_diff  weight     se   dse  warning scale
Branched     0  -1080.62   4.84       0.00     1.0  54.38  0.00    False   log
Linear       1  -1083.37   6.03       2.75     0.0  54.09  0.69     True   log
